In [309]:
import numpy as np
import itertools
from scipy.special import comb
from scipy.linalg import subspace_angles
from scipy.optimize import minimize
from sklearn.cluster import SpectralClustering

In [47]:
def single_cPCA(X:np.ndarray, Y:np.ndarray, alpha, k):
    Cx = np.cov(X.T)
    Cy = np.cov(Y.T)

    C = Cx - alpha * Cy
    _, V = np.linalg.eigh(C)

    return V[:, -k:]

In [179]:
def affinity(combination):
    combination = combination.reshape(2, d, k)
    return np.prod(np.cos(subspace_angles(*combination)))

def v_affinity(combinations):
    return np.apply_along_axis(lambda combination: affinity(combination), 1, combinations)

In [291]:
def cPCA(X:np.ndarray, Y:np.ndarray, alpha_set, k, p):
    # check d value:
    assert X.shape[1] == Y.shape[1]

    d = X.shape[1]
    V_set = []

    # calculate cPCA for each alpha
    for alpha in alpha_set:
        V = single_cPCA(X, Y, alpha, k)
        V_set.append(V)
    V_set = np.array(V_set)

    # calculate affinity Matrix for each pair of Vs
    combination_indices = np.array(list(itertools.combinations_with_replacement(range(len(V_set)), 2)))
    combinations = V_set[combination_indices].reshape(len(combination_indices), 2 * d * k)
    aff = v_affinity(combinations)

    rows = combination_indices[:, 0]
    cols = combination_indices[:, 1]
    D = np.zeros((len(V_set), len(V_set)))
    D[rows, cols] = D[cols, rows] = aff

    # cluster Vs with respect to affinities
    clustering = SpectralClustering(n_clusters=p, affinity='precomputed').fit(D)

    # find best V and alpha in each cluster
    V_star = np.zeros((p, d, k))
    alpha_star = np.zeros(p)
    for cluster in range(p):
        cluster_members = np.where(clustering.labels_ == cluster)[0]
        sums = np.sum(D[cluster_members, :][:, cluster_members], axis=1)
        V_star[cluster] = V_set[cluster_members[np.argmax(sums)]]
        alpha_star[cluster] = alpha_set[cluster_members[np.argmax(sums)]]
    
    return alpha_star, V_star

In [296]:
n = 1000
m = 100
d = 50
k = 20
p = 2

In [297]:
X = np.random.uniform(-10, 10, (n, d))
X.shape

(1000, 50)

In [298]:
Y = np.random.uniform(-10, 10, (m, d))
Y.shape

(100, 50)

In [302]:
alpha_set = np.arange(0.01, 1, 0.03)
alpha_set.shape

(33,)

In [306]:
alpha_star, V_star = cPCA(X, Y, alpha_set, k, p)

In [307]:
V_star.shape

(2, 50, 20)

In [308]:
alpha_star.shape

(2,)